# <center>Детектирование медицинских масок с использованием *YOLOv8*

<center><img src='./imgs/DSPRCV_Proj2_1_1.png'></img></center>

[Ноутбук с использованием модели *RetinaNet*](retinanet.ipynb)

## <center>Подготовка датасета
Датасет был скачан с [<i><b>Kaggle</b></i>](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection)

Импортируем необходимые библиотеки

In [1]:
from ultralytics import YOLO
from ultralytics.utils.ops import xyxy2xywh
import torch
import shutil
import os
import re
from data_classes.custom_dataset import DetectionDataset
os.chdir('..')
ANNOTATIONS_PATH = './data/annotations/'
IMAGES_PATH = './data/images/'
YOLO_PATH = './datasets/yolo_data/'
LABEL_MAP = {
    'with_mask': 0,
    'without_mask': 1,
    'mask_weared_incorrect': 2
}
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available accelerator: {DEVICE}\n')
if DEVICE == 'cuda':
    !nvidia-smi

Available accelerator: cuda

Fri Feb  9 18:20:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   45C    P8             31W /  370W |    1575MiB /  12288MiB |     33%      Default |
|                                         |                        |                  N/A |
+------------------

Создадим датафреймы для каждой выборки *train*, *valid*, *test*. Конвертируем координаты из формата $\{x_{min}, y_{min}, x_{max}, y_{max}\}$ в формат $\{x_{c}, y_{c}, w, h\}$, а также нормализуем полученные координаты, т.к. это необходимо для модели *YOLO*

Для формирования и разделения выборок используется [<b>класс датасета</b>](./data_classes/custom_dataset.py)

In [2]:
train_dataset = DetectionDataset(IMAGES_PATH, ANNOTATIONS_PATH, split='train')
train_df, full_df = train_dataset.split_df.copy(), train_dataset.full_df.copy()
valid_df = DetectionDataset(IMAGES_PATH, ANNOTATIONS_PATH, split='val').split_df.copy()
test_df = DetectionDataset(IMAGES_PATH, ANNOTATIONS_PATH, split='test').split_df.copy()
for df in [train_df, valid_df, test_df, full_df]:
    df[['xcenter', 'ycenter', 'box_width', 'box_height']] = xyxy2xywh(df[['xmin', 'ymin', 'xmax', 'ymax']].values)
    df['xcenter'] = df['xcenter'] / df['width']
    df['ycenter'] = df['ycenter'] / df['height']
    df['box_width'] = df['box_width'] / df['width']
    df['box_height'] = df['box_height'] / df['height']
    df['label'] = df['label'].map(LABEL_MAP)

full_df.head()

,name,label,width,height,xmin,ymin,xmax,ymax,xcenter,ycenter,box_width,box_height
0,maksssksksss603.png,0,400,278,49.0,11.0,55.0,17.0,0.13000,0.050360,0.0150,0.021583
1,maksssksksss603.png,0,400,278,32.0,52.0,40.0,60.0,0.09000,0.201439,0.0200,0.028777
2,maksssksksss603.png,1,400,278,52.0,76.0,61.0,86.0,0.14125,0.291367,0.0225,0.035971
3,maksssksksss603.png,0,400,278,47.0,98.0,58.0,109.0,0.13125,0.372302,0.0275,0.039568
4,maksssksksss603.png,0,400,278,68.0,100.0,75.0,111.0,0.17875,0.379496,0.0175,0.039568


Сделаем правильный формат датасета для модели *YOLO*.

In [3]:
for sample, df in [('train', train_df), ('val', valid_df), ('test', test_df)]:
    sample_path_imgs = os.path.join(YOLO_PATH, 'images', sample)
    sample_path_labels = os.path.join(YOLO_PATH, 'labels', sample)
    if not os.path.exists(sample_path_imgs):
        os.makedirs(sample_path_imgs)
    if not os.path.exists(sample_path_labels):
        os.makedirs(sample_path_labels)
    unique_images = df['name'].unique()
    for img in unique_images:
        img_df = full_df[full_df['name'] == img]
        labels_string = ''
        for row in img_df.iterrows():
            row_series = row[1]
            row_label = f'{row_series.label} {row_series.xcenter} {row_series.ycenter} {row_series.box_width} {row_series.box_height}\n'
            labels_string += row_label
        shutil.copy(os.path.join(IMAGES_PATH, img), os.path.join(sample_path_imgs))
        txt_name = re.sub(r'\.+[a-z]*', '.txt', img)
        with open(os.path.join(sample_path_labels, txt_name), 'w') as txt_label:
            txt_label.write(labels_string.rstrip('\n'))

Проверим, что датасет сформирован верно

In [4]:
!tree ./datasets -d

./datasets
└── yolo_data
    ├── images
    │   ├── test
    │   ├── train
    │   └── val
    └── labels
        ├── test
        ├── train
        └── val

9 directories


## <center>Обучение модели
Обучим (*fine-tuning*) модель с суффиксом `l` (*Large*) на 50 эпохах

[<b>masks.yaml</b>](./voc.yaml) - файл, необходимый для конфигурации модели: путь к выборкам, количество классов и их расшифровка

In [5]:
yolo = YOLO('yolov8l.pt')
results = yolo.train(
    model='yolov8l.pt',
    data='masks.yaml',
    epochs=50,
    imgsz=512, 
    batch=8,
    pretrained=True,
    visualize=True
)

New https://pypi.org/project/ultralytics/8.1.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.10 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=masks.yaml, epochs=50, time=None, patience=50, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

2024-02-09 17:15:36.169488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 17:15:36.169520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 17:15:36.205563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 26.2MB/s]


AMP: checks passed ✅


...

      50/50      4.08G     0.7969     0.3683     0.9068         21        512:  67%|██████▋   | 62/93 [00:07<00:03,  7.89it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
      50/50      4.08G     0.7849     0.3598     0.9057         28        512: 100%|██████████| 93/93 [00:11<00:00,  8.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  18%|█▊        | 4/22 [00:00<00:01, 10.41it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:02<00:00, 10.09it/s]

                   all        347       2809      0.963      0.926      0.962      0.728



50 epochs completed in 0.200 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 87.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 87.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.10 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  23%|██▎       | 5/22 [00:01<00:03,  4.60it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  7.12it/s]


                   all        347       2809      0.963      0.926      0.962      0.728
             with_mask        347       2228       0.97      0.952      0.979      0.749
          without_mask        347        506      0.934      0.902      0.956      0.692
 mask_weared_incorrect        347         75      0.986      0.925      0.952      0.743
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


Отобразим, как менялись целевые метрики и функции потерь в зависимости от эпохи

<img src="imgs/results.png" width="1280"/>

Загрузим лучшую модель (не последнюю) на основе показателей метрик/функции потерь **валидационной выборки**

In [2]:
best_model_path = './runs/detect/train/weights/best.pt'
best_yolo = YOLO(best_model_path)

## <center>Оценка качества
Сделаем валидацию на **тестовой выборке** и отобразим графики

In [3]:
test_metrics = best_yolo.val(split='test')

Ultralytics YOLOv8.1.10 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /home/deep_learning/Detection of medical masks/datasets/yolo_data/labels/test.cache... 350 images, 0 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   5%|▍         | 1/22 [00:01<00:31,  1.49s/it]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  95%|█████████▌| 21/22 [00:04<00:00,  6.79it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:05<00:00,  4.38it/s]


                   all        350       2904       0.96       0.91      0.959      0.726
             with_mask        350       2313      0.972       0.95       0.98       0.75
          without_mask        350        520      0.924      0.904      0.954      0.698
 mask_weared_incorrect        350         71      0.984      0.875      0.945      0.731
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val3


<img src="imgs/confusion_matrix_normalized.png" width="1280"/>

По матрице ошибок видно, что модель почти не путается между классами (малое количество *FP* и *FN*), однако наблюдается та же особенность, что и у модели *RetinaNet* - `background` определяется как `with_mask`

<img src="imgs/F1_curve.png" width="1280"/>

Наибольшее значение $F_1$-score ($0.94$) находится около уровня уверенности $\approx 0.41$

<img src="imgs/PR_curve.png" width="1280"/>

$\text{mAP@50} \approx 0.96$ 

Отобразим несколько тестовых примеров

<img src='imgs/val_batch0_pred.jpg'></img>